In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#vars
train_dir = "./data/"
train_set = "processed_train_data_2.csv" #train_data_polar.csv
train_path = train_dir + train_set

In [3]:
data = pd.read_csv(train_path)

In [4]:
data.head()

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,compartment_axon,compartment_basal,compartment_oblique,compartment_soma,pre_angle,post_angle,nucleus_dist,test_score_diff,oracle_diff,rf_dist
0,42593,1187660,411978,1089020,1187390,412220,1089160,304.185,353043.0,1182170.0,...,1,0,0,0,0.001596,0.002060,185207.121418,-0.189277,0.034922,340.234956
1,42594,1204580,682542,873138,1204640,682870,873890,725.431,244156.0,914243.0,...,1,0,0,0,0.003519,0.004140,185207.121418,-0.189277,0.034922,340.234956
2,42595,1191790,403683,1093180,1188590,402414,1092660,3423.030,363829.0,1171820.0,...,1,0,0,0,0.010978,0.017791,185207.121418,-0.189277,0.034922,340.234956
3,42596,1184320,419286,1082930,1186620,419721,1085540,3442.390,344267.0,1192340.0,...,1,0,0,0,0.014778,0.015690,185207.121418,-0.189277,0.034922,340.234956
4,42597,1189150,673302,944202,1188790,677771,942901,4442.380,313630.0,788566.0,...,1,0,0,0,0.027954,0.010196,185207.121418,-0.189277,0.034922,340.234956


# Feature Selection

In [169]:
# important_features = ['adp_dist','axonal_theta', 'axonal_r', 'dendritic_theta', 'dendritic_r',
#                       , 'pre_skeletal_distance_to_soma', 'post_skeletal_distance_to_soma', 
#                       'axonal_coor_z', 'dendritic_coor_z']

important_features = ['ID', 'connected','adp_dist','post_angle', 'pre_angle', 'post_skeletal_distance_to_soma', 'pre_nucleus_y',
                      'nucleus_dist', 'axonal_coor_y', 
                      'dendritic_coor_y', 'pre_nucleus_x', 'post_nucleus_y']

In [164]:
data = data[important_features]

In [5]:
data.shape

(185832, 34)

# Split and Standardize Data

In [5]:
X = data.drop(["ID", "connected"], axis=1, inplace=False)
y = data['connected']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_stand = scaler.fit_transform(X_train)
X_test_stand = scaler.transform(X_test)

In [8]:
from imblearn.over_sampling import SMOTE

In [9]:
smote = SMOTE(random_state=42)
X_train_stand_rs, y_train_rs = smote.fit_resample(X_train_stand, y_train)

In [10]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X_train_stand_rs, y_train_rs, test_size=0.2, stratify=y_train_rs, random_state=42)

full_train_x = np.concatenate([X_train, X_val], axis=0)
full_train_y = np.concatenate([y_train, y_val], axis=0)


# Modeling

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBClassifier

#metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix

In [12]:
RAND_STATE = 42

# Modeling - Random Search

## Logistic Regression

In [15]:
#param grid
lr_grid = {
    "penalty": ["l2", "elasticnet"],
    "C": np.arange(0, 1.1, 0.4),
    "l1_ratio": np.arange(0, 1.1, 4)
}

In [17]:
lr = LogisticRegression(class_weight="balanced", solver="saga", max_iter=500, random_state=RAND_STATE)

In [ ]:
lr_rs = RandomizedSearchCV(lr, param_distributions=lr_grid, cv=10, n_iter=10, scoring='balanced_accuracy')
lr_rs.fit(full_train_x, full_train_y)

/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:489: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = 1.0 / C
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:318: RuntimeWarning: invalid value encountered in double_scalars
  if step_size * alpha_scaled == 1:
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear

/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:492: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = (1.0 / C) * (1 - l1_ratio)
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:493: RuntimeWarning: divide by zero encountered in double_scalars
  beta = (1.0 / C) * l1_ratio
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:493: RuntimeWarning: invalid value encountered in double_scalars
  beta = (1.0 / C) * l1_ratio
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:318: RuntimeWarning: invalid value encountered in double_scalars
  if step_size * alpha_scaled == 1:
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:492: RuntimeWarning: divide by zero encounter

/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/anthonyyan/opt/anaconda3/envs/stackoverfit/lib/python3.10/site-packages/sklearn/l

In [ ]:
lr_rs.best_params_

In [ ]:
best_lr = lr_rs.best_estimator_

In [ ]:
lr_coef = pd.DataFrame({"weight": best_lr.coef_[0]})
lr_coef["coef"] = X.columns
lr_coef = lr_coef[["coef", "weight"]]
lr_coef.sort_values(by="weight", ascending=False)

In [ ]:
best_lr_pred = best_lr.predict(X_test_stand)

In [ ]:
lr_accuracy = accuracy_score(y_test, best_lr_pred)
lr_balanced_accuracy = balanced_accuracy_score(y_test, best_lr_pred)

In [ ]:
print("Logistic Regression Accuracy: " + str(lr_accuracy))
print("Logistic Regression Balanced Accuracy: " + str(lr_balanced_accuracy))

In [ ]:
confusion_matrix(y_test, best_lr_pred)

## Random Forest

In [75]:
#param grid
rf_grid = {
    "n_estimators": [25, 50, 100, 200],
    "criterion": ["gini", "entropy", "log_loss"],
    "max_features": [3, "sqrt", 7],
}

In [76]:
rf = RandomForestClassifier(class_weight="balanced", random_state=RAND_STATE)

In [77]:
rf_rs = RandomizedSearchCV(rf, param_distributions=rf_grid, cv=5, n_iter=10, scoring='balanced_accuracy')
rf_rs.fit(X_train_stand_rs, y_train_rs)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    random_state=42),
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_features': [3, 'sqrt', 7],
                                        'n_estimators': [25, 50, 100, 200]},
                   scoring='balanced_accuracy')

In [78]:
rf_rs.best_params_

{'n_estimators': 100, 'max_features': 3, 'criterion': 'log_loss'}

In [ ]:
best_rf = rf_rs.best_estimator_
rf_coef = pd.DataFrame({"weight": best_rf.coef_[0]})
rf_coef["coef"] = X.columns
rf_coef = rf_coef[["coef", "weight"]]
rf_coef.sort_values(by="weight", ascending=False)

In [80]:
best_rf_pred = best_rf.predict(X_test_stand)

rf_accuracy = accuracy_score(y_test, best_rf_pred)
rf_balanced_accuracy = balanced_accuracy_score(y_test, best_rf_pred)

print("Random Forest Accuracy: " + str(rf_accuracy))
print("Random Forest Balanced Accuracy: " + str(rf_balanced_accuracy))

Random Forest Accuracy: 0.9897220652729571
Random Forest Balanced Accuracy: 0.5130663909733677


In [81]:
confusion_matrix(y_test, best_rf_pred)

array([[36777,   117],
       [  265,     8]])

# Adaboost

In [83]:
#param grid
ab_grid = {
    "learning_rate": [0.001, 0.01, 0.1, 0.25, 0.5, 1.0],
    "n_estimators": [50, 100]
}

In [84]:
ab = AdaBoostClassifier(random_state=RAND_STATE)

In [ ]:
ab_rs = RandomizedSearchCV(ab, param_distributions=ab_grid, cv=5, n_iter=10, scoring='balanced_accuracy')
ab_rs.fit(X_train_stand_rs, y_train_rs)

In [ ]:
best_ab = ab_rs.best_estimator_
ab_rs.best_params_

In [ ]:
best_ab_pred = best_ab.predict(X_test_stand)

ab_accuracy = accuracy_score(y_test, best_ab_pred)
ab_balanced_accuracy = balanced_accuracy_score(y_test, best_ab_pred)

print("AdaBoost Accuracy: " + str(ab_accuracy))
print("AdaBoost Balanced Accuracy: " + str(ab_balanced_accuracy))

# Gradient Boosting

In [16]:
gb_grid = {
    "learning_rate": [0.001, 0.01, 0.1, 0.25, 0.5, 1.0],
    "n_estimators": [100, 200, 500],
    'max_features': [3, 5, 7]
}

In [17]:
gb = GradientBoostingClassifier(random_state=RAND_STATE)

In [18]:
gb_rs = RandomizedSearchCV(gb, param_distributions=gb_grid, cv=5, n_iter=10, scoring='balanced_accuracy')
gb_rs.fit(X_train_stand_rs, y_train_rs)

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   param_distributions={'learning_rate': [0.001, 0.01, 0.1,
                                                          0.25, 0.5, 1.0],
                                        'max_features': [3, 5, 7],
                                        'n_estimators': [100, 200, 500]},
                   scoring='balanced_accuracy')

In [19]:
best_gb = gb_rs.best_estimator_
gb_rs.best_params_

{'n_estimators': 200, 'max_features': 7, 'learning_rate': 1.0}

In [21]:
best_gb_pred = best_gb.predict(X_test_stand)

gb_accuracy = accuracy_score(y_test, best_gb_pred)
gb_balanced_accuracy = balanced_accuracy_score(y_test, best_gb_pred)

print("Gradient Boosting Accuracy: " + str(gb_accuracy))
print("Gradient Boosting Balanced Accuracy: " + str(gb_balanced_accuracy))

Gradient Boosting Accuracy: 0.9223235666047838
Gradient Boosting Balanced Accuracy: 0.6063742459091297


In [26]:
gb_coef = pd.DataFrame({"importance": best_gb.feature_importances_})
gb_coef["coef"] = X.columns
gb_coef = gb_coef[["coef", "importance"]]
gb_coef.sort_values(by="importance", ascending=False)

,coef,importance
6,adp_dist,0.377758
17,pre_nucleus_x,0.085725
7,post_skeletal_distance_to_soma,0.074416
27,same_brain_area,0.053857
4,dendritic_coor_y,0.045864
21,post_nucleus_y,0.033828
24,avg_post_feature_weights,0.028791
18,pre_nucleus_y,0.026650
20,post_nucleus_x,0.021622
13,post_oracle,0.021294


# SVM

In [12]:
sv_grid = {
    "C": [0.1, 0.25, 0.5, 1.0],
    "kernel": ["rbf", "linear", "poly"]
}

In [13]:
sv = SVC(class_weight="balanced", random_state=RAND_STATE)

In [ ]:
sv_rs = RandomizedSearchCV(sv, param_distributions=sv_grid, cv=5, n_iter=10, scoring='balanced_accuracy')
sv_rs.fit(X_train_stand_rs, y_train_rs)

In [ ]:
best_sv = sv_rs.best_estimator_
sv_rs.best_params_

In [ ]:
best_sv_pred = best_sv.predict(X_test_stand)

sv_accuracy = accuracy_score(y_test, best_sv_pred)
sv_balanced_accuracy = balanced_accuracy_score(y_test, best_sv_pred)

print("SVC Accuracy: " + str(sv_accuracy))
print("SVC Balanced Accuracy: " + str(sv_balanced_accuracy))

# Naive Bayes

In [ ]:
nb_grid = {
    "alpha": [0.0, 0.1, 0.25, 0.5, 0.75,1.0]
}

In [ ]:
nb = MultinomialNB(class_prior=[0.99,0.01])

In [ ]:
nb_rs = RandomizedSearchCV(nb, param_distributions=nb_grid, cv=5, n_iter=6, scoring='balanced_accuracy')
nb_rs.fit(X_train_stand_rs, y_train_rs)

In [ ]:
best_nb = nb_rs.best_estimator_
nb_rs.best_params_

In [ ]:
best_nb_pred = best_nb.predict(X_test_stand)

nb_accuracy = accuracy_score(y_test, best_nb_pred)
nb_balanced_accuracy = balanced_accuracy_score(y_test, best_nb_pred)

print("Naive Bayes Accuracy: " + str(nb_accuracy))
print("Naive Bayes Balanced Accuracy: " + str(nb_balanced_accuracy))

# Modeling - Manual Validation

## Random Forest

In [153]:
#param grid
rf_grid = {
    "n_estimators": [200, 400],
    "criterion": ["gini", "entropy", "log_loss"],
    "max_features": [7, 10, 15],
    "max_depth": [3, 5, 7]
}

In [154]:
best_rf_acc = 0.0
best_rf_model = None
best_rf_params = None
low_rf_acc = float('inf')
low_rf_model = None
low_rf_params = None
all_accs = []
for estimators in rf_grid["n_estimators"]:
#     for criterion in rf_grid["criterion"]:
    for max_features in rf_grid['max_features']:
        for max_depth in rf_grid["max_depth"]:
            print(f"Training on {estimators}, {max_features}, {max_depth}...")
            rf = RandomForestClassifier(class_weight="balanced", random_state=RAND_STATE,
                                       n_estimators=estimators, criterion="gini", max_features=max_features,
                                       max_depth = max_depth)
            rf.fit(X_train, y_train)
            val_pred = rf.predict(X_val)
            acc = balanced_accuracy_score(y_val, val_pred)
            
            if acc > 0.95:
                print(f"overfit: {acc}")
                continue
            elif acc > best_rf_acc:
                print(f"New best accuracy: {acc}")
                best_rf_acc = acc
                best_rf_model = rf
                best_rf_params = (estimators, max_features, max_depth)
            elif acc < low_rf_acc:
                print(f"New low accuracy: {acc}")
                low_rf_acc = acc
                low_rf_model = rf
                low_rf_params = (estimators, max_features,max_depth)
            else:
                print(f"Didn't improve: {acc}")


Training on 200, 7, 3...
New best accuracy: 0.7968647134685869
Training on 200, 7, 5...
New best accuracy: 0.8138901098467143
Training on 200, 7, 7...
New best accuracy: 0.8366075421546715
Training on 200, 10, 3...
New low accuracy: 0.7985418468902012
Training on 200, 10, 5...
Didn't improve: 0.8120944041337991
Training on 200, 10, 7...
Didn't improve: 0.8355572482701324
Training on 200, 15, 3...
New low accuracy: 0.7955941948767619
Training on 200, 15, 5...
Didn't improve: 0.8094517225699278
Training on 200, 15, 7...
Didn't improve: 0.8359469162473978
Training on 400, 7, 3...
New low accuracy: 0.7925957654438978
Training on 400, 7, 5...
Didn't improve: 0.8131108525278752
Training on 400, 7, 7...
Didn't improve: 0.8359976718554629
Training on 400, 10, 3...
Didn't improve: 0.7983216290707129
Training on 400, 10, 5...
Didn't improve: 0.8116878289088474
Training on 400, 10, 7...
Didn't improve: 0.8356250161648004
Training on 400, 15, 3...
Didn't improve: 0.796339629090481
Training on 400,

In [159]:
best_rf_params

(200, 7, 7)

---

## Retrain with best params on full train set

In [17]:
rf_retrained_a = RandomForestClassifier(class_weight="balanced", random_state=RAND_STATE,
                                       n_estimators=200, criterion="gini", max_features=7,
                                       max_depth = 7)
rf_retrained_a.fit(full_train_x, full_train_y)

RandomForestClassifier(class_weight='balanced', max_depth=7, max_features=7,
                       n_estimators=200, random_state=42)

In [13]:
rf_retrained_b = RandomForestClassifier(class_weight="balanced", random_state=RAND_STATE,
                                       n_estimators=200, criterion="gini", max_features=3,
                                       max_depth = 4)
rf_retrained_b.fit(full_train_x, full_train_y)

RandomForestClassifier(class_weight='balanced', max_depth=4, max_features=3,
                       n_estimators=200, random_state=42)

In [15]:
rf_retrained_c = RandomForestClassifier(class_weight="balanced", random_state=RAND_STATE,
                                       n_estimators=200, criterion="gini", max_features=5,
                                       max_depth = 5)
rf_retrained_c.fit(full_train_x, full_train_y)

RandomForestClassifier(class_weight='balanced', max_depth=5, max_features=5,
                       n_estimators=200, random_state=42)

In [20]:
rf_coef = pd.DataFrame({"weight": rf_retrained_a.feature_importances_})
rf_coef["coef"] = X.columns
rf_coef = rf_coef[["coef", "weight"]]
rf_coef.sort_values(by="weight", ascending=False)

,coef,weight
6,adp_dist,0.411951
22,compartment_axon,0.153780
27,post_angle,0.117764
26,pre_angle,0.080377
7,post_skeletal_distance_to_soma,0.073222
23,compartment_basal,0.032310
11,pre_nucleus_y,0.023790
10,pre_nucleus_x,0.015119
28,nucleus_dist,0.013352
1,axonal_coor_y,0.009777


---

### Eval

In [18]:
#best rf
best_rf_pred = rf_retrained_a.predict(X_test_stand)

rf_accuracy = accuracy_score(y_test, best_rf_pred)
rf_balanced_accuracy = balanced_accuracy_score(y_test, best_rf_pred)

print("Random Forest Accuracy: " + str(rf_accuracy))
print("Random Forest Balanced Accuracy: " + str(rf_balanced_accuracy))

Random Forest Accuracy: 0.7228724406059138
Random Forest Balanced Accuracy: 0.7531518868728171


In [14]:

best_rf_pred = rf_retrained_b.predict(X_test_stand)

rf_accuracy = accuracy_score(y_test, best_rf_pred)
rf_balanced_accuracy = balanced_accuracy_score(y_test, best_rf_pred)

print("Random Forest Accuracy: " + str(rf_accuracy))
print("Random Forest Balanced Accuracy: " + str(rf_balanced_accuracy))

Random Forest Accuracy: 0.7153119702962305
Random Forest Balanced Accuracy: 0.7402539321143973


In [16]:
best_rf_pred = rf_retrained_c.predict(X_test_stand)

rf_accuracy = accuracy_score(y_test, best_rf_pred)
rf_balanced_accuracy = balanced_accuracy_score(y_test, best_rf_pred)

print("Random Forest Accuracy: " + str(rf_accuracy))
print("Random Forest Balanced Accuracy: " + str(rf_balanced_accuracy))

Random Forest Accuracy: 0.7079667446928727
Random Forest Balanced Accuracy: 0.7456438909927282


# XGradient Boosting

In [17]:
#param grid
gb_grid = {
    "booster": ['gbtree'],
    "eta": [0.001, 0.005],
    "max_depth": [3, 4, 5],
    'colsample_bytree': [0.1, 0.2, 0.25, 0.3],
    'subsample': [0.1, 0.2, 0.25, 0.3],
}

In [27]:
best_gb_acc = 0.0
best_gb_model = None
best_gb_params = None
low_gb_acc = float('inf')
low_gb_model = None
low_gb_params = None
all_params = []
all_models = []
for booster in gb_grid["booster"]:
    for eta in gb_grid["eta"]:
        for max_depth in gb_grid["max_depth"]:
            for colsample in gb_grid["colsample_bytree"]:
                for subsample in gb_grid["subsample"]:
#                     for gamma in gb_grid["gamma"]:
#                         for min_child_weight in gb_grid["min_child_weight"]:
                    print(f"Training on {booster}, {eta}, {max_depth}, {colsample}, {subsample}...")
                    gb = XGBClassifier(booster=booster, eta=eta,
                                               max_depth = 3, colsample_bytree=colsample,
                                      gamma=5.0,min_child_weight=3, subsample=subsample)
                    gb.fit(X_train, y_train)
                    val_pred = gb.predict(X_val)
                    acc = balanced_accuracy_score(y_val, val_pred)
                    all_params.append((booster, eta, max_depth, colsample, subsample, gamma, min_child_weight))
                    all_models.append(gb)
                    if acc > 0.90:
                        print(f"overfit: {acc}")
                        continue
                    elif acc > best_gb_acc:
                        print(f"New best accuracy: {acc}")
                        best_gb_acc = acc
                        best_gb_model = gb
                        best_gb_params = (booster, eta, max_depth, colsample, subsample)
                    elif acc < low_gb_acc:
                        print(f"New low accuracy: {acc}")
                        low_gb_acc = acc
                        low_gb_model = gb
                        low_gb_params = (booster, eta, max_depth, colsample, subsample)
                    else:
                        print(f"Didn't improve: {acc}")
            

Training on gbtree, 0.001, 3, 0.1, 0.1...
New best accuracy: 0.8249178569843147
Training on gbtree, 0.001, 3, 0.1, 0.2...
New best accuracy: 0.8255107959285446
Training on gbtree, 0.001, 3, 0.1, 0.25...
New best accuracy: 0.8255785678410947
Training on gbtree, 0.001, 3, 0.1, 0.3...
New low accuracy: 0.8248670198706423
Training on gbtree, 0.001, 3, 0.2, 0.1...
New best accuracy: 0.8258669467337093
Training on gbtree, 0.001, 3, 0.2, 0.2...
Didn't improve: 0.8253756579095319
Training on gbtree, 0.001, 3, 0.2, 0.25...
Didn't improve: 0.824918261642436
Training on gbtree, 0.001, 3, 0.2, 0.3...
Didn't improve: 0.8250029631880154
Training on gbtree, 0.001, 3, 0.25, 0.1...
New low accuracy: 0.8226652377241335
Training on gbtree, 0.001, 3, 0.25, 0.2...
New low accuracy: 0.8223094749315787
Training on gbtree, 0.001, 3, 0.25, 0.25...
New low accuracy: 0.8219367848019272
Training on gbtree, 0.001, 3, 0.25, 0.3...
Didn't improve: 0.8223264258019845
Training on gbtree, 0.001, 3, 0.3, 0.1...
Didn't i

In [28]:
for i in range(len(all_models)):
    pre = all_models[i].predict(X_test_stand)
    acc = balanced_accuracy_score(y_test, pre)
    print(f"{i}/{len(all_models)}: balanced accuracy: {acc}")
    if acc == best_gb_acc:
        print("BEST ACCURACY ^^")
    elif acc == low_gb_acc:
        print("LOWEST ACCURACY ^^")
        

0/96: balanced accuracy: 0.7317721038651271
1/96: balanced accuracy: 0.7372259523422313
2/96: balanced accuracy: 0.7352724794585259
3/96: balanced accuracy: 0.7304420385815735
4/96: balanced accuracy: 0.7440350347327092
5/96: balanced accuracy: 0.7418918291011314
6/96: balanced accuracy: 0.7423661609708121
7/96: balanced accuracy: 0.7422983992751435
8/96: balanced accuracy: 0.7475954278279859
9/96: balanced accuracy: 0.7404727055889847
10/96: balanced accuracy: 0.7474463520975149
11/96: balanced accuracy: 0.7404862579281184
12/96: balanced accuracy: 0.7407030953542582
13/96: balanced accuracy: 0.7422364457248178
14/96: balanced accuracy: 0.7350053047727466
15/96: balanced accuracy: 0.7370536440303882
16/96: balanced accuracy: 0.7317721038651271
17/96: balanced accuracy: 0.7372259523422313
18/96: balanced accuracy: 0.7352724794585259
19/96: balanced accuracy: 0.7304420385815735
20/96: balanced accuracy: 0.7440350347327092
21/96: balanced accuracy: 0.7418918291011314
22/96: balanced accu

In [22]:
best_gb_ee = all_models[0] 
best_gb_ee

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.25, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.001, eval_metric=None,
              feature_types=None, gamma=5.0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [21]:
best_gb_params_ee = all_params[0]
best_gb_params_ee = ('gbtree', 0.001, 3, 0.25, 0.25, 5.0, 3) #for reproducibility

In [20]:
best_gb_test_acc = 0.7474463520975149

In [23]:
gb_retrained_a = XGBClassifier(booster='gbtree', eta=best_gb_params_ee[1],max_depth = 3, 
                             colsample_bytree=best_gb_params_ee[3],
                             gamma=best_gb_params_ee[5],min_child_weight=best_gb_params_ee[6], 
                             subsample=best_gb_params_ee[4])
gb_retrained_a.fit(full_train_x, full_train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.25, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.001, eval_metric=None,
              feature_types=None, gamma=5.0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

### Eval

In [24]:
best_gb_pred = gb_retrained_a.predict(X_test_stand)

gb_accuracy = accuracy_score(y_test, best_gb_pred)
gb_balanced_accuracy = balanced_accuracy_score(y_test, best_gb_pred)

print("Random Forest Accuracy: " + str(gb_accuracy))
print("Random Forest Balanced Accuracy: " + str(gb_balanced_accuracy))

Random Forest Accuracy: 0.7121909220545107
Random Forest Balanced Accuracy: 0.738681860774884


# Save Model

In [66]:
import pickle

In [29]:
#name of best...
    #rf: rf_retrained_a
    #xgb: gb_retrained_a
model_to_save_submit = gb_retrained_a
model_name = "xgb"
month = "12"
day = "1"

In [13]:
lr_file = "lr_initial.sav"

In [99]:
pickle.dump(model_to_save_submit, open(lr_file, 'wb'))

# Submit

In [30]:
lb_data = pd.read_csv("./data/processed_leaderboard_data_2.csv")

In [31]:
lb_data.head()

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,compartment_axon,compartment_basal,compartment_oblique,compartment_soma,pre_angle,post_angle,nucleus_dist,test_score_diff,oracle_diff,rf_dist
0,0,527499,544761,912282,529457,543459,916958,4783.37,239163.00,178032.0,...,0,0,0,0,0.005807,0.021848,867856.993086,0.094901,-0.018742,410.307333
1,1,1269890,471870,837816,1271230,469651,841045,4060.72,294573.00,1193910.0,...,1,0,0,0,0.033029,0.036083,186530.704389,0.011774,0.006508,429.207435
2,2,666057,469875,925239,665387,467214,923430,2912.80,4102.75,387714.0,...,0,1,0,0,0.004614,0.270614,701103.615692,0.111555,-0.173209,429.195374
3,3,696633,453516,925806,696648,453417,921875,3272.83,31123.00,424714.0,...,0,0,0,0,0.005799,0.112964,701103.615692,0.111555,-0.173209,429.195374
4,4,567525,585921,888426,570714,583676,891264,4441.75,114201.00,250411.0,...,0,1,0,0,0.004202,0.029444,717601.628226,0.103745,-0.044866,427.279840


In [32]:
X_exam = lb_data.drop(["ID"], inplace=False, axis=1)
X_exam_stand = scaler.transform(X_exam)

In [33]:
exam_pred = model_to_save_submit.predict(X_exam_stand)

In [39]:
# lb_data['connected'] = exam_pred
lb_data['connected_'] = lb_data['connected'] > 0.5

In [40]:
submit_df = lb_data.filter(['ID', 'connected'])

In [41]:
submit_df.head()

,ID,connected
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


In [42]:
submit_name = f"{model_name}_{month}_{day}"
submit_df.to_csv(f'./submissions/{submit_name}.csv',index=False)
